<a href="https://colab.research.google.com/github/akashapare/bosch-classification/blob/main/mobile_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import tensorflow as tf
from tensorflow import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image,ImageFilter,ImageStat,ImageTransform, ImageOps,ImageShow
import random
from tensorflow.keras.metrics import categorical_accuracy, top_k_categorical_accuracy, categorical_crossentropy
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input
#start = dt.datetime.now()

In [2]:
import zipfile
import os

zip_ref = zipfile.ZipFile('/content/drive/MyDrive/dataset.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('/content') #Extracts the files into the /tmp folder
zip_ref.close()


In [3]:
INPUT_DIR = '/content/dataset'
#INPUT_DIR = '/content/drive/MyDrive/edge dataset'
#INPUT_DIR = '/content/drive/MyDrive/gray dataset'
#INPUT_DIR = '/content/drive/MyDrive/gray_eq dataset'

In [3]:
BASE_SIZE =512 
NCATS = 5
tf.random.set_seed(1996)

In [7]:



EPOCHS = 50
size = 512
batchsize = 50

#model for colour images

In [8]:
model = MobileNet(input_shape=(size, size, 3), alpha=1., weights=None, classes=NCATS)
model.compile(optimizer=Adam(lr=0.002), loss='categorical_crossentropy',
              metrics=['accuracy'])
print(model.summary())

Model: "mobilenet_1.00_512"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 512, 512, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 256, 256, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 256, 256, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 256, 256, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 256, 256, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 256, 256, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 256, 256, 32

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [9]:
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',validation_split=0.3,)
validation_generator=datagen.flow_from_directory(INPUT_DIR,
                            target_size=(size,size),
                            color_mode='rgb',
                            class_mode='categorical',
                            batch_size=batchsize,
                            seed=1996,
                            subset='validation')
train_generator=datagen.flow_from_directory(INPUT_DIR,
                            target_size=(size,size),
                            color_mode='rgb',
                            class_mode='categorical',
                            batch_size=batchsize,
                            seed=1996,
                            subset='training')

Found 1026 images belonging to 5 classes.
Found 2403 images belonging to 5 classes.


In [10]:
from sklearn.utils import class_weight 
class_weights = class_weight.compute_class_weight(
           'balanced',
            np.unique(train_generator.classes), 
            train_generator.classes)
class_weights= dict(enumerate(class_weights))
model.fit_generator(train_generator,
    steps_per_epoch = train_generator.samples // batchsize,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batchsize,
    epochs = EPOCHS,
    class_weight=class_weights)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Epoch 1/500


ResourceExhaustedError: ignored

In [17]:
model.save('my_model_color')

INFO:tensorflow:Assets written to: my_model_color/assets


#model for grayscale/edge images

In [5]:
model = MobileNet(input_shape=(size, size, 1), alpha=1., weights=None, classes=5)
model.compile(optimizer=Adam(lr=0.002), loss='categorical_crossentropy',
              metrics=['accuracy'])
print(model.summary())

Model: "mobilenet_1.00_512"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 512, 512, 1)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 256, 256, 32)      288       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 256, 256, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 256, 256, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 256, 256, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 256, 256, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 256, 256, 32

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [6]:
INPUT_DIR = '/content/drive/MyDrive/edge dataset'
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',validation_split=0.3)
validation_generator=datagen.flow_from_directory(INPUT_DIR,
                            target_size=(size,size),
                            color_mode='grayscale',
                            class_mode='categorical',
                            batch_size=batchsize,
                            seed=1996,
                            subset='validation')
train_generator=datagen.flow_from_directory(INPUT_DIR,
                            target_size=(size,size),
                            color_mode='grayscale',
                            class_mode='categorical',
                            batch_size=batchsize,
                            seed=1996,
                            subset='training')

Found 1026 images belonging to 5 classes.
Found 2403 images belonging to 5 classes.


In [8]:
from sklearn.utils import class_weight 
class_weights = class_weight.compute_class_weight(
           'balanced',
            np.unique(train_generator.classes), 
            train_generator.classes)
class_weights= dict(enumerate(class_weights))
checkpoint_filepath = '/content/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
model.fit_generator(train_generator,
    steps_per_epoch = train_generator.samples // batchsize,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batchsize,
    epochs = EPOCHS,
    class_weight=class_weights,callbacks=[model_checkpoint_callback],workers=4)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Epoch 1/50
48/48 [==============================] - 280s 5s/step - loss: 1.2515 - accuracy: 0.3519 - val_loss: 1.7718 - val_accuracy: 0.0700
Epoch 2/50
48/48 [==============================] - 105s 2s/step - loss: 0.8222 - accuracy: 0.5593 - val_loss: 1.6548 - val_accuracy: 0.0710
Epoch 3/50
48/48 [==============================] - 105s 2s/step - loss: 0.6363 - accuracy: 0.6634 - val_loss: 1.6313 - val_accuracy: 0.0710
Epoch 4/50
48/48 [==============================] - 103s 2s/step - loss: 0.5659 - accuracy: 0.6893 - val_loss: 1.5659 - val_accuracy: 0.6120
Epoch 5/50
48/48 [==============================] - 107s 2s/step - loss: 0.4855 - accuracy: 0.7556 - val_loss: 1.6424 - val_accuracy: 0.6120
Epoch 6/50
48/48 [==============================] - 106s 2s/step - loss: 0.4148 - accuracy: 0.7756 - val_loss: 1.5478 - val_accuracy: 0.6200
Epoch 7/50
48/48

In [9]:
model.save('my_model_gray')

INFO:tensorflow:Assets written to: my_model_gray/assets
